In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
Data=pd.read_csv('../input/used-car-dataset-ford-and-mercedes/merc.csv')

In [ ]:
Data.head()

In [ ]:
Data.info()

In [ ]:
Data.isna().sum()

## Data Analysis

In [ ]:
print('Model unique values: ',len(Data['model'].unique()))
print(Data['model'].unique())

In [ ]:
plt.figure(figsize=(25,8))
sns.countplot(Data['model'])
plt.show()

In [ ]:
plt.figure(figsize=(25,8))
sns.countplot(Data['year'])
plt.show()

In [ ]:
print('Transmission unique: ',len(Data['transmission'].unique()))
print(Data['transmission'].unique())

In [ ]:
sns.countplot(Data['transmission'])
plt.show()

In [ ]:
print('Fuelt type unique values: ',len(Data['fuelType'].unique()))
print(Data['fuelType'].unique())

In [ ]:
sns.countplot(Data['fuelType'])
plt.show()

In [ ]:
Data['engineSize'].value_counts()

In [ ]:
plt.figure(figsize=(13,8))
sns.countplot(Data['engineSize'])
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(Data['tax'])
plt.show()

In [ ]:
Data['price'].describe()

## Find values

In [ ]:
Data['price'][Data['price']>100000]

In [ ]:
Data['fuelType'][Data['fuelType']=='Other']

In [ ]:
Data['year'][Data['year']<2000]

## Data Preprocessing

In [ ]:
New_Data=Data.copy().drop(index=Data['price'][Data['price']>100000].index.tolist()+Data['fuelType'][Data['fuelType']=='Other'].index.tolist()+Data['year'][Data['year']<2000].index.tolist()+Data['transmission'][Data['transmission']=='Other'].index.tolist())

In [ ]:
New_Data.info()

In [ ]:
data_model_dummies=pd.get_dummies(New_Data['model'])

In [ ]:
data_model_dummies

In [ ]:
transmission_dummies=pd.get_dummies(New_Data['transmission'])

In [ ]:
transmission_dummies

In [ ]:
fueltype_dummies=pd.get_dummies(New_Data['fuelType'])

In [ ]:
fueltype_dummies

In [ ]:
New_Data=New_Data.drop(columns=(['fuelType','transmission','model']))  

In [ ]:
New_Data.head()

In [ ]:
New_Data=pd.concat(([data_model_dummies,New_Data.iloc[:,:2],transmission_dummies,New_Data.iloc[:,2:3],fueltype_dummies,New_Data.iloc[:,3:]]),axis=1)

In [ ]:
New_Data

In [ ]:
x=New_Data.copy().drop(columns='price')

In [ ]:
x.head()

In [ ]:
y=New_Data.copy().pop('price')

In [ ]:
y.head()

## Train - Validation-Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_array=x.values
y_array=y.values

In [ ]:
x_array

In [ ]:
y_array

### validation

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_array,y_array,test_size=0.10,random_state=42) 

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_val.shape

In [ ]:
y_val.shape 

### Test

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.20,random_state=42)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

## Standart Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
obje_ss=StandardScaler()

x_train_ss=obje_ss.fit_transform(x_train)
x_val_ss=obje_ss.fit_transform(x_val)
x_test_ss=obje_ss.fit_transform(x_test)

In [ ]:
x_train

In [ ]:
x_train_ss

## Models

### LinearRegression

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model_le=LinearRegression().fit(x_train,y_train)
model_le

In [ ]:
y_pred=model_le.predict(x_val)

In [ ]:
print('Train success rate : %',model_le.score(x_train,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred))

In [ ]:
y_pred[y_pred<0] # Linearregression outliner data predict is so bad  

### SupportVectorRegression

In [ ]:
from sklearn.svm import SVR

In [ ]:
model_svr=SVR(kernel='linear',degree=3).fit(x_train_ss,y_train)
model_svr

In [ ]:
y_pred=model_svr.predict(x_val_ss) 

In [ ]:
print('Train success rate : %',model_svr.score(x_train_ss,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

### PolynomialRegression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
obje_poly=PolynomialFeatures(degree=2)

x_train_poly=obje_poly.fit_transform(x_train)
x_val_poly=obje_poly.fit_transform(x_val)
x_test_poly=obje_poly.fit_transform(x_test)

In [ ]:
model_poly=LinearRegression(n_jobs=-1).fit(x_train_poly,y_train)
model_poly

In [ ]:
y_pred=model_poly.predict(x_val_poly)

In [ ]:
print('Train success rate : %',model_poly.score(x_train_poly,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

In [ ]:
y_pred[y_pred<0] #Outliner Data 

### RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model_rfr=RandomForestRegressor(n_estimators=100,n_jobs=-1,random_state=42).fit(x_train_ss,y_train)
model_rfr

In [ ]:
y_pred=model_rfr.predict(x_val_ss)

In [ ]:
print('Train success rate : %',model_rfr.score(x_train_ss,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

### AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
model_ada=AdaBoostRegressor(n_estimators=10,random_state=42).fit(x_train_ss,y_train)
model_ada

In [ ]:
y_pred=model_ada.predict(x_val_ss) 

In [ ]:
print('Train success rate : %',model_ada.score(x_train_ss,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

### XGBoostRegressor

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model_xgb=XGBRegressor(max_depth=3,learning_rate=0.1,n_estimators=100,verbosity=0,random_state=42).fit(x_train,y_train)
model_xgb

In [ ]:
y_pred=model_xgb.predict(x_val) 

In [ ]:
print('Train success rate : %',model_xgb.score(x_train,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

### LightGBM

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
model_lgb=LGBMRegressor(max_depth=2,learning_rate=0.5,n_estimators=100,random_state=42).fit(x_train,y_train)
model_lgb

In [ ]:
y_pred=model_lgb.predict(x_val)

In [ ]:
print('Train success rate : %',model_lgb.score(x_train,y_train)*100)
print('Validation success rate : %',r2_score(y_val,y_pred)*100)

## Test

In [ ]:
def test_models(model_name):

  for i in model_name:
    print(f'{i.__class__} | Test success rate : % {r2_score(y_test,i.predict(x_test))*100}')

def test_models_ss(model_name):

   for a in model_name:
    print(f'{a.__class__} Test success rate : % {r2_score(y_test,a.predict(x_test_ss))*100}')

In [ ]:
model_names=[model_le,model_xgb,model_lgb]
model_names_ss=[model_rfr,model_svr,model_ada]

In [ ]:
test_models(model_names)

In [ ]:
test_models_ss(model_names_ss) 

## Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
model_names=[model_le,model_rfr,model_ada,model_xgb,model_lgb]

In [ ]:
def test_models_cross(model_name):

  for i in model_name:
    print(f'{i.__class__} | Test(Cross validation) success rate : % {cross_val_score(i,X=x_train,y=y_train,cv=5,n_jobs=-1).mean()*100}')

def test_models_ss_cross(model_name):

   for a in model_name:
    print(f'{a.__class__} | Test(Cross validation)-ss success rate : % {cross_val_score(a,X=x_train_ss,y=y_train,cv=5,n_jobs=-1).mean()*100}')

In [ ]:
test_models_cross(model_names)

In [ ]:
test_models_ss_cross(model_names)

In [ ]:
print('Support vector regression cross-val success rate:',cross_val_score(model_svr,X=x_train_ss,y=y_train,cv=3,n_jobs=-1).mean()*100)

In [ ]:
print('Polynomial regression cross-val success rate:',cross_val_score(model_poly,X=x_train_ss,y=y_train,cv=3,n_jobs=-1).mean()*100)